In [39]:
import datetime
import os
import re

import pandas as pd

from multiplex_imaging_pipeline.utils import listfiles
from multiplex_imaging_pipeline.utils import get_ome_tiff_channels

In [40]:
"""
https://gist.github.com/wassname/1393c4a57cfcbf03641dbc31886123b8
"""

import unicodedata
import string

valid_filename_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
char_limit = 255

def clean_filename(filename, whitelist=valid_filename_chars, replace=' '):
    # replace spaces
    for r in replace:
        filename = filename.replace(r,'_')
    
    # keep only valid ascii chars
    cleaned_filename = unicodedata.normalize('NFKD', filename).encode('ASCII', 'ignore').decode()
    
    # keep only whitelisted chars
    cleaned_filename = ''.join(c for c in cleaned_filename if c in whitelist)
    if len(cleaned_filename)>char_limit:
        print("Warning, filename truncated because it was over {}. Filenames may no longer be unique".format(char_limit))
    return cleaned_filename[:char_limit]    


In [41]:
# omero stuff
group = 'HTAN'
project = 'CODEX'

In [42]:
initial = pd.read_csv('/diskmnt/Projects/Users/estorrs/sandbox/codex_tracking_v1.tsv',
                     sep='\t', index_col=0)
initial

,omero_run_name,region,Date of experiment,Name of person who performed experiment,Project,katmai qptiff filepath,storage1 qptiff filepath,katmai ome tiff filepath,Case name Specimen 1,Specimen 1 name,...,Location on slide Specimen 8,Specimen 8 bbox,Case name Specimen 9,Specimen 9 name,Location on slide Specimen 9,Specimen 9 bbox,Case name Specimen 10,Specimen 10 name,Location on slide Specimen 10,Specimen 10 bbox
Run Name,,,,,,,,,,,,,,,,,,,,,
20240206_HS-13-6668_2B_HS-16-40491_1D,NaN,NaN,02/06/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240206_HS-13-666...,NaN,NaN,HS-13-6668_2B,HS-13-6668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed,NaN,NaN,02/02/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240205_HS-12-309...,NaN,NaN,HS-12-30984,HS-12-30984_1F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240201_HS-13-23034_1C_HS-14-13304_1C,2024-02-01_HS-13-23034__HS-14-13304,NaN,02/01/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240201_HS-13-230...,NaN,/diskmnt/primary/CODEX/HTAN/20240201_HS-13-230...,HS-13-23034,HS-13-23034_1C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240126_Human_Liver_SenNet_SN009H1_SN046H1_SN047H1_SN001H1,2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1,NaN,01/26/2024,Xiang Li,HTAN,/diskmnt/primary/CODEX/HTAN/20240126_Human_Liv...,NaN,/diskmnt/primary/CODEX/HTAN/20240126_Human_Liv...,SN009H1,SN009H1-Md1Fp2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240125_Human_Liver_SenNet_SN010H1_SN007H1_SN018H1,2024-01-25_SN007H1__SN010H1__SN018H1__SN018H1-...,NaN,01/25/2024,Xiang Li,HTAN,/diskmnt/primary/CODEX/HTAN/20240125_Human_Liv...,NaN,/diskmnt/primary/CODEX/HTAN/20240125_Human_Liv...,SN010H1,SN010H1-Md1Fp2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,NaN,10/19/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,HT339B1,HT339B1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,NaN,10/08/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,HT323B1,HT323B1-S1H3A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,reg_001,09/21/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,HT293N1,HT293N1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# initial = initial.iloc[1:]

In [44]:
dirs = [x.split('/')[5] for x in initial['katmai qptiff filepath'].to_list()]
dirs

['20240206_HS-13-6668_2B_HS-16-40491_1D',
 '20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed',
 '20240201_HS-13-23034_1C_HS-14-13304_1C',
 '20240126_Human_Liver_SenNet_SN009H1_SN046H1_SN047H1_SN001H1',
 '20240125_Human_Liver_SenNet_SN010H1_SN007H1_SN018H1',
 '20240118-C3L-02551-11',
 '20240117-C3L-00982-11',
 '20240111_Human_mCRC_serial_sectrion_HT413C1_Th1k4A1_Slide_8',
 '20240110_Human_mCRC_Serial_section_HT413C1_Th1k4A1_Slide3',
 '20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00733-12U2-PA00002352U1_Btom',
 '20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N-01200-11Us1_2_middle_C3N-01200-13-U2_btm',
 '20231208_Human_CRC_Primary_Bob_group_S123TRF_Slide6',
 '20231207_Human_CRC_Primary_Bob_group_S110TLF_Slide6',
 '20231123_Human_pancreatic_cancer_TMA_UNST_TW17-2886-132',
 '20231122_Human_pancreatic_cancer_S18-25943-A7Us1_2__Us1_9__Us1_13__Us1_16',
 '20231118_Human_prostate_African_American_serial_S18_5591_Slide_6',
 '20231116_Human_prostate_African_American_serial_S18_5591_Slide_2',
 '2023

In [45]:
# redo run dates
roots = [re.sub(r'^[0-9\/]*[-_]*(.*)$', r'\1', x) for x in dirs]
roots

['HS-13-6668_2B_HS-16-40491_1D',
 'HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed',
 'HS-13-23034_1C_HS-14-13304_1C',
 'Human_Liver_SenNet_SN009H1_SN046H1_SN047H1_SN001H1',
 'Human_Liver_SenNet_SN010H1_SN007H1_SN018H1',
 'C3L-02551-11',
 'C3L-00982-11',
 'Human_mCRC_serial_sectrion_HT413C1_Th1k4A1_Slide_8',
 'Human_mCRC_Serial_section_HT413C1_Th1k4A1_Slide3',
 'Human_CCRCC_C3L-00970-13u2_top_C3N-00733-12U2-PA00002352U1_Btom',
 'Human_CCRCC_C3L-01287-11Us1_2_top_C3N-01200-11Us1_2_middle_C3N-01200-13-U2_btm',
 'Human_CRC_Primary_Bob_group_S123TRF_Slide6',
 'Human_CRC_Primary_Bob_group_S110TLF_Slide6',
 'Human_pancreatic_cancer_TMA_UNST_TW17-2886-132',
 'Human_pancreatic_cancer_S18-25943-A7Us1_2__Us1_9__Us1_13__Us1_16',
 'Human_prostate_African_American_serial_S18_5591_Slide_6',
 'Human_prostate_African_American_serial_S18_5591_Slide_2',
 'SenNet_TreatBRCA_HT378B1_HT088B1',
 'SenNet_Liver_SN039H1_SN004H1_SN032H1',
 'Prostate_Primary_vs_Met_S21_34912A2_S21_53957A1_S19_41530D1_S22_53426A1',
 'Pro

In [46]:
dates = []
for x in initial['Date of experiment']:
    month, day, year = x.split('/')
    date = datetime.date(int(year), int(month), int(day))
    date_str = str(date)
    dates.append(date_str)
dates

['2024-02-06',
 '2024-02-02',
 '2024-02-01',
 '2024-01-26',
 '2024-01-25',
 '2024-01-18',
 '2024-01-17',
 '2024-01-11',
 '2024-01-10',
 '2023-12-20',
 '2023-12-19',
 '2023-12-08',
 '2023-12-07',
 '2023-11-23',
 '2023-11-22',
 '2023-11-18',
 '2023-11-16',
 '2023-11-04',
 '2023-11-02',
 '2023-10-21',
 '2023-10-19',
 '2023-10-07',
 '2023-10-06',
 '2023-10-02',
 '2023-09-14',
 '2023-09-13',
 '2023-08-30',
 '2023-08-29',
 '2023-08-26',
 '2023-08-24',
 '2023-08-23',
 '2023-08-10',
 '2023-08-03',
 '2023-07-29',
 '2023-07-28',
 '2023-07-20',
 '2023-07-19',
 '2023-07-01',
 '2023-06-23',
 '2023-06-20',
 '2023-06-10',
 '2023-06-06',
 '2023-06-05',
 '2023-05-26',
 '2023-04-20',
 '2023-01-13',
 '2023-01-08',
 '2023-04-13',
 '2023-04-20',
 '2023-04-27',
 '2023-04-13',
 '2023-03-15',
 '2023-04-07',
 '2023-03-16',
 '2023-04-19',
 '2023-04-28',
 '2023-04-22',
 '2023-04-14',
 '2023-05-12',
 '2023-03-07',
 '2023-03-08',
 '2023-03-04',
 '2023-02-23',
 '2023-02-17',
 '2023-02-21',
 '2023-02-08',
 '2023-02-

In [47]:
run_strings = []
case_cols = [c for c in initial.columns if 'Case name' in c]
for i, row in initial.iterrows():
    row = row[case_cols]
    vals = [clean_filename(val) for val in row.to_list() if not pd.isnull(val)]
    vals = sorted(set(vals))
    run_strings.append('__'.join(vals))
run_strings

['HS-13-6668_2B__HS-16-40491',
 'HS-12-30984__HS-13-18789',
 'HS-13-23034__HS-14-13304',
 'SN001H1__SN009H1__SN046H1__SN047H1',
 'SN007H1__SN010H1__SN018H1',
 'C3L-02551',
 'C3L-00982',
 'HT413C1_Th1k4A1',
 'HT413C1_Th1k4A1',
 'C3L-00970__C3N-00733__PA00002352',
 'C3L-01287__C3N-01200',
 'S123TRF',
 'S110TLF',
 '',
 'S18-25943',
 'S18-5591',
 'S18-5591',
 'HT088B1__HT378B1',
 'SN004H1__SN032H1__SN039H1',
 'S19_41530__S21_34912__S21_53957__S22_53426',
 'S21_11816__S21_44455',
 'S18_11798__S18_15142__S18_18215',
 'S18-9906',
 'HT206B1',
 'HT206B1',
 'HT425B1__HT591B1',
 'C3L-01046__C3L-02041__C3N-01197',
 'HS-16-21554__HS-17-12425',
 'S15-32775',
 'S16-38794',
 'S16-38794',
 'C3N-03070__C3N-03448__C3N-03450',
 'P1_F1LM3l__P21_F2RM6R',
 'E165_F4LM2L__P1_F2LM3l__P21_F4RM5R',
 'P1_F1LM3l__P21_F2RM6R',
 'E165_F6RM3R__P1_F1LM3l__P21_F2RM6R',
 'E165_F4LM2L__P1_F2LM3l__P21_F4RM5R',
 'HT206B1__HT297B1__HT339B2__HT565B1',
 'K2300120-3__K2300125-2__K2300132-3',
 'pkd_14504__pkd_14508__pkd_811__pkd

In [48]:
runs = [f'{date}_{run}' for date, run in zip(dates, run_strings)]
runs

['2024-02-06_HS-13-6668_2B__HS-16-40491',
 '2024-02-02_HS-12-30984__HS-13-18789',
 '2024-02-01_HS-13-23034__HS-14-13304',
 '2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1',
 '2024-01-25_SN007H1__SN010H1__SN018H1',
 '2024-01-18_C3L-02551',
 '2024-01-17_C3L-00982',
 '2024-01-11_HT413C1_Th1k4A1',
 '2024-01-10_HT413C1_Th1k4A1',
 '2023-12-20_C3L-00970__C3N-00733__PA00002352',
 '2023-12-19_C3L-01287__C3N-01200',
 '2023-12-08_S123TRF',
 '2023-12-07_S110TLF',
 '2023-11-23_',
 '2023-11-22_S18-25943',
 '2023-11-18_S18-5591',
 '2023-11-16_S18-5591',
 '2023-11-04_HT088B1__HT378B1',
 '2023-11-02_SN004H1__SN032H1__SN039H1',
 '2023-10-21_S19_41530__S21_34912__S21_53957__S22_53426',
 '2023-10-19_S21_11816__S21_44455',
 '2023-10-07_S18_11798__S18_15142__S18_18215',
 '2023-10-06_S18-9906',
 '2023-10-02_HT206B1',
 '2023-09-14_HT206B1',
 '2023-09-13_HT425B1__HT591B1',
 '2023-08-30_C3L-01046__C3L-02041__C3N-01197',
 '2023-08-29_HS-16-21554__HS-17-12425',
 '2023-08-26_S15-32775',
 '2023-08-24_S16-38794',
 '2

In [49]:
counts = {}
for i, run in enumerate(runs):
    if run not in counts:
        counts[run] = 1
    else:
        count = counts[run] + 1
        counts[run] = count
        runs[i] = f'{run}_{count}'
runs

['2024-02-06_HS-13-6668_2B__HS-16-40491',
 '2024-02-02_HS-12-30984__HS-13-18789',
 '2024-02-01_HS-13-23034__HS-14-13304',
 '2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1',
 '2024-01-25_SN007H1__SN010H1__SN018H1',
 '2024-01-18_C3L-02551',
 '2024-01-17_C3L-00982',
 '2024-01-11_HT413C1_Th1k4A1',
 '2024-01-10_HT413C1_Th1k4A1',
 '2023-12-20_C3L-00970__C3N-00733__PA00002352',
 '2023-12-19_C3L-01287__C3N-01200',
 '2023-12-08_S123TRF',
 '2023-12-07_S110TLF',
 '2023-11-23_',
 '2023-11-22_S18-25943',
 '2023-11-18_S18-5591',
 '2023-11-16_S18-5591',
 '2023-11-04_HT088B1__HT378B1',
 '2023-11-02_SN004H1__SN032H1__SN039H1',
 '2023-10-21_S19_41530__S21_34912__S21_53957__S22_53426',
 '2023-10-19_S21_11816__S21_44455',
 '2023-10-07_S18_11798__S18_15142__S18_18215',
 '2023-10-06_S18-9906',
 '2023-10-02_HT206B1',
 '2023-09-14_HT206B1',
 '2023-09-13_HT425B1__HT591B1',
 '2023-08-30_C3L-01046__C3L-02041__C3N-01197',
 '2023-08-29_HS-16-21554__HS-17-12425',
 '2023-08-26_S15-32775',
 '2023-08-24_S16-38794',
 '2

In [50]:
for run in runs: print(run)

2024-02-06_HS-13-6668_2B__HS-16-40491
2024-02-02_HS-12-30984__HS-13-18789
2024-02-01_HS-13-23034__HS-14-13304
2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1
2024-01-25_SN007H1__SN010H1__SN018H1
2024-01-18_C3L-02551
2024-01-17_C3L-00982
2024-01-11_HT413C1_Th1k4A1
2024-01-10_HT413C1_Th1k4A1
2023-12-20_C3L-00970__C3N-00733__PA00002352
2023-12-19_C3L-01287__C3N-01200
2023-12-08_S123TRF
2023-12-07_S110TLF
2023-11-23_
2023-11-22_S18-25943
2023-11-18_S18-5591
2023-11-16_S18-5591
2023-11-04_HT088B1__HT378B1
2023-11-02_SN004H1__SN032H1__SN039H1
2023-10-21_S19_41530__S21_34912__S21_53957__S22_53426
2023-10-19_S21_11816__S21_44455
2023-10-07_S18_11798__S18_15142__S18_18215
2023-10-06_S18-9906
2023-10-02_HT206B1
2023-09-14_HT206B1
2023-09-13_HT425B1__HT591B1
2023-08-30_C3L-01046__C3L-02041__C3N-01197
2023-08-29_HS-16-21554__HS-17-12425
2023-08-26_S15-32775
2023-08-24_S16-38794
2023-08-23_S16-38794
2023-08-10_C3N-03070__C3N-03448__C3N-03450
2023-08-03_P1_F1LM3l__P21_F2RM6R
2023-07-29_E165_F4LM2L__P1

In [51]:
assert len(runs) == len(set(runs))

In [52]:
initial['omero_run_name'] = runs

In [53]:
initial

,omero_run_name,region,Date of experiment,Name of person who performed experiment,Project,katmai qptiff filepath,storage1 qptiff filepath,katmai ome tiff filepath,Case name Specimen 1,Specimen 1 name,...,Location on slide Specimen 8,Specimen 8 bbox,Case name Specimen 9,Specimen 9 name,Location on slide Specimen 9,Specimen 9 bbox,Case name Specimen 10,Specimen 10 name,Location on slide Specimen 10,Specimen 10 bbox
Run Name,,,,,,,,,,,,,,,,,,,,,
20240206_HS-13-6668_2B_HS-16-40491_1D,2024-02-06_HS-13-6668_2B__HS-16-40491,NaN,02/06/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240206_HS-13-666...,NaN,NaN,HS-13-6668_2B,HS-13-6668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed,2024-02-02_HS-12-30984__HS-13-18789,NaN,02/02/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240205_HS-12-309...,NaN,NaN,HS-12-30984,HS-12-30984_1F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240201_HS-13-23034_1C_HS-14-13304_1C,2024-02-01_HS-13-23034__HS-14-13304,NaN,02/01/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240201_HS-13-230...,NaN,/diskmnt/primary/CODEX/HTAN/20240201_HS-13-230...,HS-13-23034,HS-13-23034_1C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240126_Human_Liver_SenNet_SN009H1_SN046H1_SN047H1_SN001H1,2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1,NaN,01/26/2024,Xiang Li,HTAN,/diskmnt/primary/CODEX/HTAN/20240126_Human_Liv...,NaN,/diskmnt/primary/CODEX/HTAN/20240126_Human_Liv...,SN009H1,SN009H1-Md1Fp2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240125_Human_Liver_SenNet_SN010H1_SN007H1_SN018H1,2024-01-25_SN007H1__SN010H1__SN018H1,NaN,01/25/2024,Xiang Li,HTAN,/diskmnt/primary/CODEX/HTAN/20240125_Human_Liv...,NaN,/diskmnt/primary/CODEX/HTAN/20240125_Human_Liv...,SN010H1,SN010H1-Md1Fp2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,NaN,10/19/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,HT339B1,HT339B1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,NaN,10/08/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,HT323B1,HT323B1-S1H3A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,reg_001,09/21/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,HT293N1,HT293N1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## make generate whole slide ome.tiff commands

In [54]:
def get_platform(fp):
    root = fp.split('/')[-1]
    
    if root.split('.')[-1] == 'qptiff':
        platform = 'phenocycler'
    elif root.split('.')[-1] == 'tif':
        platform = 'codex'
    else:
        platform = 'raw'
    
    return platform

In [55]:
ome_cmds = []
omero_cmds = []
ome_fps = []

skip = [
    '/diskmnt/primary/CODEX/HTAN/20231006_Prostate_Serial_S18-9906_slide_3/S18-9906#3_Scan1.qptiff'
]
for i, row in initial.iterrows():
    qptiff = row['katmai qptiff filepath']
    if qptiff not in skip:
        assert qptiff.replace('/', '') == clean_filename(qptiff), f"{qptiff}"
    qptiff = re.sub(r'/$', r'', qptiff)

    run = row['omero_run_name']
    platform = get_platform(qptiff)

    prefix = '/'.join(qptiff.split('/')[:-1])
        
    out_fp = os.path.join(prefix, f'{run}.ome.tiff')
    ome_fps.append(out_fp)
    
    ome_cmd = f'mip make-ome --input-tif {qptiff} --platform {platform} --output-filepath {out_fp}'
    
    omero_cmd = f'omero import {out_fp} -T Project:name:{project}/Dataset:name:{run}'
    
    ome_cmds.append(ome_cmd)
    omero_cmds.append(omero_cmd)
    

In [56]:
for fp in ome_fps:
    print(fp)

/diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/2024-02-06_HS-13-6668_2B__HS-16-40491.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/2024-02-02_HS-12-30984__HS-13-18789.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240201_HS-13-23034_1C_HS-14-13304_1C/2024-02-01_HS-13-23034__HS-14-13304.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240126_Human_Liver_SenNet_SN009H1_SN046H1_SN047H1_SN001H1/2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240125_Human_Liver_SenNet_SN010H1_SN007H1_SN018H1/2024-01-25_SN007H1__SN010H1__SN018H1.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240118-C3L-02551-11/2024-01-18_C3L-02551.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240117-C3L-00982-11/2024-01-17_C3L-00982.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240111_Human_mCRC_serial_sectrion_HT413C1_Th1k4A1_Slide_8/2024-01-11_HT413C1_Th1k4A1.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240110_Human_mCRC_Serial_section_HT413C1_Th1k4A1_Slide3/2024-01-10_HT

In [57]:
for cmd in ome_cmds:
    print(cmd)

mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/20240206_HS-13-6668_2B_HS-16-40491_1D.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/2024-02-06_HS-13-6668_2B__HS-16-40491.ome.tiff
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/HS-12-30984_1F_HS-13-18789_1B_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/2024-02-02_HS-12-30984__HS-13-18789.ome.tiff
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240201_HS-13-23034_1C_HS-14-13304_1C/20240201_HS-13-23034_1C_HS-14-13304_1C_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240201_HS-13-23034_1C_HS-14-13304_1C/2024-02-01_HS-13-23034__HS-14-13304.ome.tiff
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240126_Human_Liver_SenNet_SN009H1_SN046H1_SN047H

In [58]:
for cmd in omero_cmds:
    print(cmd)

omero import /diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/2024-02-06_HS-13-6668_2B__HS-16-40491.ome.tiff -T Project:name:CODEX/Dataset:name:2024-02-06_HS-13-6668_2B__HS-16-40491
omero import /diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/2024-02-02_HS-12-30984__HS-13-18789.ome.tiff -T Project:name:CODEX/Dataset:name:2024-02-02_HS-12-30984__HS-13-18789
omero import /diskmnt/primary/CODEX/HTAN/20240201_HS-13-23034_1C_HS-14-13304_1C/2024-02-01_HS-13-23034__HS-14-13304.ome.tiff -T Project:name:CODEX/Dataset:name:2024-02-01_HS-13-23034__HS-14-13304
omero import /diskmnt/primary/CODEX/HTAN/20240126_Human_Liver_SenNet_SN009H1_SN046H1_SN047H1_SN001H1/2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1.ome.tiff -T Project:name:CODEX/Dataset:name:2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1
omero import /diskmnt/primary/CODEX/HTAN/20240125_Human_Liver_SenNet_SN010H1_SN007H1_SN018H1/2024-01-25_SN007H1__SN010H1__SN018H1.ome.tiff -T Project:name:CODEX/

In [31]:
assert len(ome_fps) == len(set(ome_fps))
assert len(ome_cmds) == len(set(ome_cmds))
assert len(omero_cmds) == len(set(omero_cmds))

In [32]:
initial['katmai ome tiff filepath'] = ome_fps

/diskmnt/Projects/Users/estorrs/sandbox/tmp/ipykernel_239926/846165846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial['katmai ome tiff filepath'] = ome_fps


In [31]:
# check for completion

for fp, ome_cmd in zip(initial['katmai ome tiff filepath'], ome_cmds):
    if not os.path.exists(fp):
#         print(fp)
        print(ome_cmd)

mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00733-12U2-PA00002352U1_Btom/20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00733-12U2-PA00002352U1_Btom.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00733-12U2-PA00002352U1_Btom/2023-12-20_C3L-00970__C3N-00733__PA00002352.ome.tiff
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N-01200-11Us1_2_middle_C3N-01200-13-U2_btm/20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N-01200-11Us1_2_middle_C3N-01200-13-U2_btm.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N-01200-11Us1_2_middle_C3N-01200-13-U2_btm/2023-12-19_C3L-01287__C3N-01200.ome.tiff
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20231208_Human_CRC_Primary_Bob_group_S123TRF_Slide6/S123TRF_Slide6_Scan1.qptiff --platform phenocycler --output-

In [22]:

for fp, ome_cmd in zip(initial['katmai ome tiff filepath'], ome_cmds):
    fs = os.stat(fp).st_size
    print(fs, fp)

FileNotFoundError: [Errno 2] No such file or directory: '/diskmnt/primary/CODEX/HTAN/20240111_Human_mCRC_serial_sectrion_HT413C1_Th1k4A1_Slide_8/2024-01-11_HT413C1_Th1k4A1.ome.tiff'

In [25]:
# check for completion

for fp, ome_cmd in zip(initial['katmai ome tiff filepath'], ome_cmds):
    if os.path.exists(fp):
        try:
            channels = get_ome_tiff_channels(fp)
            if not channels:
#                 print(fp)
                print(ome_cmd)
        except:
#             print(fp)
            print(ome_cmd)

/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## make generate specimen specific ome.tiff commands

In [59]:
initial

,omero_run_name,region,Date of experiment,Name of person who performed experiment,Project,katmai qptiff filepath,storage1 qptiff filepath,katmai ome tiff filepath,Case name Specimen 1,Specimen 1 name,...,Location on slide Specimen 8,Specimen 8 bbox,Case name Specimen 9,Specimen 9 name,Location on slide Specimen 9,Specimen 9 bbox,Case name Specimen 10,Specimen 10 name,Location on slide Specimen 10,Specimen 10 bbox
Run Name,,,,,,,,,,,,,,,,,,,,,
20240206_HS-13-6668_2B_HS-16-40491_1D,2024-02-06_HS-13-6668_2B__HS-16-40491,NaN,02/06/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240206_HS-13-666...,NaN,NaN,HS-13-6668_2B,HS-13-6668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed,2024-02-02_HS-12-30984__HS-13-18789,NaN,02/02/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240205_HS-12-309...,NaN,NaN,HS-12-30984,HS-12-30984_1F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240201_HS-13-23034_1C_HS-14-13304_1C,2024-02-01_HS-13-23034__HS-14-13304,NaN,02/01/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240201_HS-13-230...,NaN,/diskmnt/primary/CODEX/HTAN/20240201_HS-13-230...,HS-13-23034,HS-13-23034_1C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240126_Human_Liver_SenNet_SN009H1_SN046H1_SN047H1_SN001H1,2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1,NaN,01/26/2024,Xiang Li,HTAN,/diskmnt/primary/CODEX/HTAN/20240126_Human_Liv...,NaN,/diskmnt/primary/CODEX/HTAN/20240126_Human_Liv...,SN009H1,SN009H1-Md1Fp2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240125_Human_Liver_SenNet_SN010H1_SN007H1_SN018H1,2024-01-25_SN007H1__SN010H1__SN018H1,NaN,01/25/2024,Xiang Li,HTAN,/diskmnt/primary/CODEX/HTAN/20240125_Human_Liv...,NaN,/diskmnt/primary/CODEX/HTAN/20240125_Human_Liv...,SN010H1,SN010H1-Md1Fp2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,NaN,10/19/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,HT339B1,HT339B1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,NaN,10/08/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,HT323B1,HT323B1-S1H3A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,reg_001,09/21/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,HT293N1,HT293N1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
ome_cmds = []
omero_cmds = []
data = []
case_cols = [c for c in initial.columns if re.findall(r'Specimen [0-9] name', c)]
fps_counts = {}
case_cols
for i, row in initial.iterrows():
    qptiff = row['katmai qptiff filepath']
    if qptiff not in skip:
        assert qptiff.replace('/', '') == clean_filename(qptiff), f'{qptiff}'
    qptiff = re.sub(r'/$', r'', qptiff)

    run = row['omero_run_name']
    platform = get_platform(qptiff)
    
    prefix = '/'.join(qptiff.split('/')[:-1])
    
    specimens = []
    for col in case_cols:
        specimen_name = row[col]
        if not pd.isnull(specimen_name):
            specimen_name = clean_filename(specimen_name)
            specimen_number = re.sub(r'^Specimen ([0-9]+) name', r'\1', col)
            case_name = row[f'Case name Specimen {specimen_number}']

            if specimen_number != '1':
                bbox = row[f'Specimen {specimen_number} bbox']
            else:
                bbox = row[f'Specimen {specimen_number} bbox (TOP,BOTTOM,LEFT,RIGHT)']
            date = run.split('_')[0].replace('-', '')
            
            out_fp = os.path.join(prefix, f'{specimen_name}__{date}.ome.tiff')
            
            if out_fp not in fps_counts:
                fps_counts[out_fp] = 1
            else:
                fps_counts[out_fp] += 1
                count = fps_counts[out_fp]
                out_fp = os.path.join(prefix, f'{specimen_name}__{count}__{date}.ome.tiff')

            if not pd.isnull(bbox):
                ome_cmd = f'mip make-ome --input-tif {qptiff} --platform {platform} --output-filepath {out_fp} --bbox {bbox}'
            else:
                ome_cmd = f'mip make-ome --input-tif {qptiff} --platform {platform} --output-filepath {out_fp}'

            omero_cmd = f'omero import {out_fp} -T Project:name:{project}/Dataset:name:{run}'

            ome_cmds.append(ome_cmd)
            omero_cmds.append(omero_cmd)

            data.append([i, run, date, case_name, specimen_name, bbox, out_fp])
            specimens.append(specimen_name)
    if len(specimens) != len(set(specimens)):
        raise RuntimeError(f'{i}, specimen names should be unique: {specimens}')

specimen_df = pd.DataFrame(
    data,
    columns=['run_name', 'omero_run_name', 'date', 'case', 'specimen', 'bbox', 'filepath'],
)
specimen_df   

,run_name,omero_run_name,date,case,specimen,bbox,filepath
0,20240206_HS-13-6668_2B_HS-16-40491_1D,2024-02-06_HS-13-6668_2B__HS-16-40491,20240206,HS-13-6668_2B,HS-13-6668,"746,13340,1106,20719",/diskmnt/primary/CODEX/HTAN/20240206_HS-13-666...
1,20240206_HS-13-6668_2B_HS-16-40491_1D,2024-02-06_HS-13-6668_2B__HS-16-40491,20240206,HS-16-40491,HS-16-40491_1D,"15979,47146,0,19979",/diskmnt/primary/CODEX/HTAN/20240206_HS-13-666...
2,20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed,2024-02-02_HS-12-30984__HS-13-18789,20240202,HS-12-30984,HS-12-30984_1F,"0,27620,0,29732",/diskmnt/primary/CODEX/HTAN/20240205_HS-12-309...
3,20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed,2024-02-02_HS-12-30984__HS-13-18789,20240202,HS-13-18789,HS-13-18789_1B,"30449,48775,2176,21108",/diskmnt/primary/CODEX/HTAN/20240205_HS-12-309...
4,20240201_HS-13-23034_1C_HS-14-13304_1C,2024-02-01_HS-13-23034__HS-14-13304,20240201,HS-13-23034,HS-13-23034_1C,"0,32536,0,23040",/diskmnt/primary/CODEX/HTAN/20240201_HS-13-230...
...,...,...,...,...,...,...,...
270,20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,20211019,HT339B1,HT339B1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...
271,20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,20211008,HT323B1,HT323B1-S1H3A4,NaN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...
272,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,20210921,HT293N1,HT293N1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...
273,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1_2,20210921,HT293N1,HT293N1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...


In [61]:
for cmd in ome_cmds: print(cmd)

mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/20240206_HS-13-6668_2B_HS-16-40491_1D.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/HS-13-6668__20240206.ome.tiff --bbox 746,13340,1106,20719
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/20240206_HS-13-6668_2B_HS-16-40491_1D.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/HS-16-40491_1D__20240206.ome.tiff --bbox 15979,47146,0,19979
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/HS-12-30984_1F_HS-13-18789_1B_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/HS-12-30984_1F__20240202.ome.tiff --bbox 0,27620,0,29732
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984

In [62]:
for cmd in omero_cmds: print(cmd)

omero import /diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/HS-13-6668__20240206.ome.tiff -T Project:name:CODEX/Dataset:name:2024-02-06_HS-13-6668_2B__HS-16-40491
omero import /diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/HS-16-40491_1D__20240206.ome.tiff -T Project:name:CODEX/Dataset:name:2024-02-06_HS-13-6668_2B__HS-16-40491
omero import /diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/HS-12-30984_1F__20240202.ome.tiff -T Project:name:CODEX/Dataset:name:2024-02-02_HS-12-30984__HS-13-18789
omero import /diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/HS-13-18789_1B__20240202.ome.tiff -T Project:name:CODEX/Dataset:name:2024-02-02_HS-12-30984__HS-13-18789
omero import /diskmnt/primary/CODEX/HTAN/20240201_HS-13-23034_1C_HS-14-13304_1C/HS-13-23034_1C__20240201.ome.tiff -T Project:name:CODEX/Dataset:name:2024-02-01_HS-13-23034__HS-14-13304
omero import /diskmnt/primary/CODEX/HTAN/20240201_HS-13

In [64]:
assert specimen_df.shape[0] == len(set(specimen_df['filepath']))
assert len(ome_cmds) == len(set(ome_cmds))
assert len(omero_cmds) == len(set(omero_cmds))

In [65]:
specimen_df.to_csv('/diskmnt/Projects/Users/estorrs/sandbox/codex_specimens.txt', sep='\t', index=False)

In [69]:
print('\t'.join(specimen_df.columns))
for i in range(specimen_df.shape[0]):
    print('\t'.join([x for x in specimen_df.iloc[i]]))

run_name	omero_run_name	date	case	specimen	bbox	filepath
20240206_HS-13-6668_2B_HS-16-40491_1D	2024-02-06_HS-13-6668_2B__HS-16-40491	20240206	HS-13-6668_2B	HS-13-6668	746,13340,1106,20719	/diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/HS-13-6668__20240206.ome.tiff
20240206_HS-13-6668_2B_HS-16-40491_1D	2024-02-06_HS-13-6668_2B__HS-16-40491	20240206	HS-16-40491	HS-16-40491_1D	15979,47146,0,19979	/diskmnt/primary/CODEX/HTAN/20240206_HS-13-6668_2B_HS-16-40491_1D/HS-16-40491_1D__20240206.ome.tiff
20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed	2024-02-02_HS-12-30984__HS-13-18789	20240202	HS-12-30984	HS-12-30984_1F	0,27620,0,29732	/diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/HS-12-30984_1F__20240202.ome.tiff
20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed	2024-02-02_HS-12-30984__HS-13-18789	20240202	HS-13-18789	HS-13-18789_1B	30449,48775,2176,21108	/diskmnt/primary/CODEX/HTAN/20240205_HS-12-30984_1F_HS-13-18789_1B_AkoyaFixed/HS-13-18789

TypeError: sequence item 5: expected str instance, float found

In [36]:
# check for completion

for i, fp, cmd in zip(specimen_df['omero_run_name'], specimen_df['filepath'], ome_cmds):
    if not os.path.exists(fp):
#         print(i, fp)
        print(cmd)

mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/01282023_PKD_K2200446_3PB/K2200446_3PB_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/01282023_PKD_K2200446_3PB/K2200446_3PB__20230128.ome.tiff


In [35]:
# check for completion

for fp, ome_cmd in zip(specimen_df['filepath'], ome_cmds):
    if os.path.exists(fp):
        try:
            channels = get_ome_tiff_channels(fp)
            if not channels:
#                 print(fp)
                print(ome_cmd)
        except:
#             print(fp)
            print(ome_cmd)

/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20230913_BRCA_HT425B1-Extra__HT591B1-S1H2__HT591B1-S1H1/HT425B1-Extra__HT591B1-S1H2__HT591B1-S1H1_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20230913_BRCA_HT425B1-Extra__HT591B1-S1H2__HT591B1-S1H1/HT425B1-Extra__20230913.ome.tiff --bbox 0,26000,0,14000
